# 🔎 The Einstein Guide to OpenSearch Query DSL

Welcome to the masterclass on OpenSearch Query DSL.

## 1. The Two Contexts: Query vs. Filter

Before writing a single line of code, understand this distinction. It is the single biggest factor in performance.

| Context | Question Asked | Scoring | Caching | Use Case |
| :--- | :--- | :--- | :--- | :--- |
| **Query** | "How well does this document match?" | Yes (Relevance Score) | No | Full-text search, "best match" |
| **Filter** | "Does this document match? (Yes/No)" | No (Score = 0) | **Yes (Bitset Cache)** | Exact values, ranges, binary decisions |

### 1.1 Query Context (Relevance Scoring)
In the query context, OpenSearch calculates a **Relevance Score** (`_score`) for each document.
*   **The Question**: "How well does this document match the query clause?"
*   **The Result**: A floating-point number (e.g., `1.5`, `0.1`). Higher is better.
*   **Use Case**: Full-text search where you want the "best" matches at the top (e.g., searching for "error" should rank "critical error" higher than "minor error").

### 1.2 Filter Context (Caching)
In the filter context, OpenSearch only checks if the document matches.
*   **The Question**: "Does the document match?" (Binary Yes/No).
*   **The Result**: No score (usually `0.0` or `1.0` constant).
*   **The Superpower**: **Caching**. OpenSearch caches the result of frequently used filters (using a bitset). This makes subsequent searches with the same filter **instant**.
*   **Use Case**:
    *   `status: "active"`
    *   `timestamp: [2023-01-01 TO 2023-01-31]`
    *   `author_id: 123`

**🚀 Efficiency Tip:** Use **Filter** context whenever you don't need a relevance score.

In [36]:
import requests
import json

# Configuration
OPENSEARCH_URL = "http://localhost:19200"
AUTH = ('admin', 'OpenSearch@2024')
VERIFY_SSL = False
INDEX_NAME = "patronidata"

def search(query=None, index=INDEX_NAME, explain=False, body=None):
    """
    Helper function to execute a search query.
    Args:
        query: The query object (will be wrapped in {"query": ...})
        index: The index to search
        explain: Whether to include explanation
        body: The FULL search body (overrides 'query'). Use this for top-level params like 'rescore'.
    """
    url = f"{OPENSEARCH_URL}/{index}/_search"
    
    if body:
        payload = body
    else:
        payload = {"query": query}
        
    if explain:
        payload["explain"] = True
        
    try:
        response = requests.post(url, json=payload, auth=AUTH, verify=VERIFY_SSL)
        response.raise_for_status()
        result = response.json()
        
        hits = result['hits']['hits']
        print(f"⚡ Found {result['hits']['total']['value']} hits in {result['took']}ms")
        print("-" * 50)
        for i, hit in enumerate(hits[:5]): # Show top 5
            source = hit['_source']
            # Try to find a meaningful field to display
            display_text = source.get('message') or source.get('log') or source.get('_raw') or str(source)[:100]
            print(f"[{i+1}] Score: {hit['_score']} | ID: {hit['_id']}")
            print(f"    Content: {display_text[:200]}...")
            print("-" * 50)
            
    except Exception as e:
        print(f"Error: {e}")
        if 'response' in locals():
            print(response.text)

print("✅ Setup Complete. Helper function `search(query=None, body=None)` is ready.")

✅ Setup Complete. Helper function `search(query=None, body=None)` is ready.


In [18]:
# Demo: Query Context vs Filter Context
# We will run the SAME requirement in both contexts to see the difference in _score.

# 1. Query Context (Calculates Score)
q_context = {
    "bool": {
        "must": [
            { "term": { "host.name": "e20666c13238" } }
        ]
    }
}
print("--- QUERY CONTEXT (Note the _score) ---")
search(q_context)

# 2. Filter Context (No Score, Cached)
f_context = {
    "bool": {
        "filter": [
            { "term": { "host.name": "e20666c13238" } }
        ]
    }
}
print("\n--- FILTER CONTEXT (Score is 0.0) ---")
search(f_context)

--- QUERY CONTEXT (Note the _score) ---
⚡ Found 10000 hits in 1ms
--------------------------------------------------
[1] Score: 0.99749315 | ID: qLdDzJoBDLC7DRstIO5B
    Content: 2025-11-28 20:59:00 UTC [39]: [22436-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[2] Score: 0.99749315 | ID: qbdDzJoBDLC7DRstIO5B
    Content: 2025-11-28 20:59:00 UTC [39]: [22437-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[3] Score: 0.99749315 | ID: qrdDzJoBDLC7DRstIO5B
    Content: 2025-11-28 20:59:00 UTC [34]: [22440-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg

## 1.3 Term vs. Full Text: The Analysis Trap

This is the most common source of "Why is my query finding nothing?" frustration.

| Feature | Term-Level Queries (`term`, `range`) | Full-Text Queries (`match`, `multi_match`) |
| :--- | :--- | :--- |
| **Analysis** | **NO**. Searches for the *exact* bytes you provide. | **YES**. Analyzes your query string (tokenizes, lowercases, stems) to match the field's analyzer. |
| **Target Field** | `keyword`, `integer`, `date`, `boolean` | `text` |
| **Case Sensitivity** | **Yes** (usually). "Error" != "error". | **No** (usually). "Error" becomes "error" during analysis. |
| **Partial Match** | No. Must match the full token exactly. | Yes (matches individual tokens). |

### ⚠️ The Trap
If you use a `term` query on a `text` field, it will look for the **exact** string you typed in the inverted index.
*   Index contains: `["fatal", "error", "occurred"]` (Standard analyzer lowercased it).
*   You search `term: "Error"` -> **FAIL** ("Error" != "error").
*   You search `match: "Error"` -> **SUCCESS** (Analyzer converts "Error" -> "error", which matches).

In [28]:
# Demo: The Analysis Trap
# We will search for "Patroni" in the '_raw' field (which is text/analyzed).

# 1. Match Query (Analyzed)
# "Patroni" -> Analyzer -> "patroni"
# Index has "patroni" -> MATCH!
print("--- MATCH Query for 'Patroni' (Analyzed) ---")
search({
    "match": { "_raw": "Patroni" }
})

# 2. Term Query (Exact)
# "Patroni" -> No Analysis -> "Patroni"
# Index has "patroni" -> "Patroni" != "patroni" -> NO MATCH!
print("\n--- TERM Query for 'Patroni' (Exact) ---")
search({
    "term": { "_raw": "Patroni" }
})

# 3. Term Query (Exact Lowercase)
# "patroni" -> No Analysis -> "patroni"
# Index has "patroni" -> MATCH! (But this is brittle, don't rely on it for text fields)
print("\n--- TERM Query for 'patroni' (Exact Lowercase) ---")
search({
    "term": { "_raw": "patroni" }
})

--- MATCH Query for 'Patroni' (Analyzed) ---
⚡ Found 10000 hits in 1ms
--------------------------------------------------
[1] Score: 0.27441108 | ID: ecz4tJoBI2mmqWwuPbKe
    Content: Traceback (most recent call last):
  File "/usr/local/lib/python3.13/dist-packages/patroni/ha.py", line 2136, in _run_cycle
    self.load_cluster_from_dcs()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/u...
--------------------------------------------------
[2] Score: 0.2700388 | ID: f8z4tJoBI2mmqWwuPbKf
    Content: Traceback (most recent call last):
  File "/usr/local/lib/python3.13/dist-packages/patroni/dcs/etcd.py", line 571, in handle_etcd_exceptions
    retval = func(self, *args, **kwargs)
  File "/usr/local...
--------------------------------------------------
[3] Score: 0.25762922 | ID: cMwDtZoBI2mmqWwuv7vK
    Content: 2025-11-24 08:38:32 UTC INFO: Got response from patroni1 http://patroni1:8008/patroni: {"state": "starting", "role": "replica", "dcs_last_seen": 1763973509, "database_system_identifie

## 2. Leaf Queries

Leaf queries look for a specific value in a specific field. They can be used alone.

### A. Full Text Queries (Analyzed)
Use these for `text` fields. The query string is **analyzed** using the same analyzer as the field.

*   **`match`**: The standard full-text search. (Operator: OR by default)
*   **`match_phrase`**: Matches exact phrase (terms in specific order).
*   **`multi_match`**: Run a match query across multiple fields.

In [14]:
# 1. Match Query (The Go-To)
# Searches for 'error' OR 'failed'. 
# If the field analyzer stems 'failed' -> 'fail', this will match 'fail'.
q_match = {
    "match": {
        "message": "error failed"
    }
}
print("--- MATCH QUERY ---")
search(q_match)

# 2. Match Phrase (Precision)
# Searches for 'connection failed' exactly in that order.
q_phrase = {
    "match_phrase": {
        "message": "connection failed"
    }
}
print("\n--- MATCH PHRASE QUERY ---")
search(q_phrase)

# 3. Multi Match (Cross-Field)
# Searches 'error' in both 'message' and '_raw' fields.
# Useful when you don't know exactly which field contains the data.
q_multi = {
    "multi_match": {
        "query": "error",
        "fields": ["message", "_raw"]
    }
}
print("\n--- MULTI MATCH QUERY ---")
search(q_multi)

--- MATCH QUERY ---
⚡ Found 1 hits in 1ms
--------------------------------------------------
[1] Score: 0.13076457 | ID: mrTOxJoBDLC7DRst4JdI
    Content: CRITICAL error: This is a test log to trigger the YAML monitor...
--------------------------------------------------

--- MATCH PHRASE QUERY ---
⚡ Found 0 hits in 0ms
--------------------------------------------------

--- MULTI MATCH QUERY ---
⚡ Found 84 hits in 12ms
--------------------------------------------------
[1] Score: 7.4890895 | ID: Ssz4tJoBI2mmqWwuFrKM
    Content: 2025-11-24 08:25:47 UTC ERROR: watchprefix failed: <Unavailable error: 'error reading from server: EOF', code: 14>...
--------------------------------------------------
[2] Score: 7.078575 | ID: xrIAvZoBDLC7DRstxC5O
    Content: 2025-11-25T21:52:15.437Z UTC [TEST]: ERROR:  58P01: Simulated disk I/O error (Class 58)...
--------------------------------------------------
[3] Score: 7.078575 | ID: f7IavZoBDLC7DRst1jah
    Content: 2025-11-25T22:20:44.064Z UTC [TES

### A.2 Advanced Full-Text Queries
Beyond the basics, these queries offer more control for specific use cases like autocomplete or complex user input.

*   **`match_phrase_prefix`**: The "Autocomplete" query. Matches a phrase where the last term is treated as a prefix.
*   **`match_bool_prefix`**: Creates a boolean query from the terms, with the last term as a prefix.
*   **`query_string`**: The "Power User" query. Supports boolean logic (`AND`, `OR`, `NOT`) directly in the string.
*   **`simple_query_string`**: The "Safe" version of `query_string`. Won't throw errors on syntax mistakes. Great for search bars.
*   **`intervals`**: Fine-grained control over order and proximity of terms.

In [24]:
# 4. Match Phrase Prefix (Autocomplete)
# Matches "connection fai" -> "connection failed"
q_prefix_phrase = {
    "match_phrase_prefix": {
        "message": "connection fai"
    }
}
print("--- MATCH PHRASE PREFIX ---")
search(q_prefix_phrase)

# 5. Match Bool Prefix
# Matches "connection" AND "fai*" (prefix)
# Difference from phrase: Terms don't need to be adjacent, just present.
q_bool_prefix = {
    "match_bool_prefix": {
        "message": "connection fai"
    }
}
print("\n--- MATCH BOOL PREFIX ---")
search(q_bool_prefix)

# 6. Query String (Power User Syntax)
# Allows users to write "error AND postgres OR (fatal AND NOT warning)"
q_string = {
    "query_string": {
        "query": "(error OR fatal) AND postgres",
        "default_field": "_raw"
    }
}
print("\n--- QUERY STRING ---")
search(q_string)

# 7. Simple Query String (Safe Search Bar)
# Users can type "error +postgres -warning" without breaking the query
q_simple = {
    "simple_query_string": {
        "query": "error +postgres -warning",
        "fields": ["message", "_raw"]
    }
}
print("\n--- SIMPLE QUERY STRING ---")
search(q_simple)

# 8. Intervals Query (Proximity Control)
# Finds 'connection' and 'failed' appearing near each other (ordered)
q_intervals = {
    "intervals": {
        "message": {
            "all_of": {
                "ordered": True,
                "intervals": [
                    { "match": { "query": "connection" } },
                    { "match": { "query": "failed" } }
                ]
            }
        }
    }
}
print("\n--- INTERVALS QUERY ---")
search(q_intervals)

--- MATCH PHRASE PREFIX ---
⚡ Found 0 hits in 1ms
--------------------------------------------------

--- MATCH BOOL PREFIX ---
⚡ Found 0 hits in 0ms
--------------------------------------------------

--- QUERY STRING ---
⚡ Found 10000 hits in 10ms
--------------------------------------------------
[1] Score: 5.7332807 | ID: gbR4xJoBDLC7DRstH3oI
    Content: 2025-11-27 08:39:56 UTC [125412]: [2-1] user=postgres,db=postgres,app=psql,client=[local] ERROR:  relation "log_test" does not exist at character 13...
--------------------------------------------------
[2] Score: 2.2581577 | ID: LMz5tJoBI2mmqWwulbR0
    Content: 2025-11-24 08:27:26 UTC [29]: [1-1] user=postgres,db=postgres,app=[unknown],client=127.0.0.1 FATAL:  the database system is starting up...
--------------------------------------------------
[3] Score: 2.2581577 | ID: Lcz5tJoBI2mmqWwulbR0
    Content: 2025-11-24 08:27:26 UTC [31]: [1-1] user=postgres,db=postgres,app=[unknown],client=127.0.0.1 FATAL:  the database system is

### B. Term Level Queries (Not Analyzed)
Use these for `keyword`, `integer`, `date`, etc. The query is **exact**. 
**⚠️ Warning:** Never use `term` on a `text` field unless you know exactly what the analyzer produced (usually lowercase tokens).

*   **`term`**: Exact match.
*   **`terms`**: Match any of the provided values (OR).
*   **`range`**: Greater than, less than, etc.
*   **`exists`**: Field is not null.

In [9]:
# 1. Term Query (Exact)
# Note: If 'host.name' is a keyword field, this must match exactly (case-sensitive).
# We found the host name 'e20666c13238' from previous inspection.
q_term = {
    "term": {
        "host.name": "e20666c13238" 
    }
}
print("--- TERM QUERY ---")
search(q_term)

# 2. Range Query (Efficient)
# Great for dates and numbers.
q_range = {
    "range": {
        "@timestamp": {
            "gte": "now-1h",
            "lt": "now"
        }
    }
}
print("\n--- RANGE QUERY ---")
search(q_range)

--- TERM QUERY ---
⚡ Found 10000 hits in 3ms
--------------------------------------------------
[1] Score: 1.0132241 | ID: bLcvzJoBDLC7DRstg-Nq
    Content: 2025-11-28 20:37:36 UTC [39]: [21582-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[2] Score: 1.0132241 | ID: bbcvzJoBDLC7DRstg-Nq
    Content: 2025-11-28 20:37:36 UTC [39]: [21583-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[3] Score: 1.0132241 | ID: brcvzJoBDLC7DRstg-Nq
    Content: 2025-11-28 20:37:36 UTC [34]: [21586-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
-------

### C. The Extended Term-Level Arsenal
Beyond simple `term` and `range`, here are other powerful tools for structured data.

*   **`terms`**: The "IN" operator. Matches any of the provided values.
*   **`terms_set`**: Matches documents that match a minimum number of terms.
*   **`ids`**: Retrieve specific documents by their `_id`.
*   **`exists`**: Finds documents where a field is present and non-null.
*   **`prefix`**: Matches terms starting with a specific string (Faster than wildcard, but still use with care).
*   **`regexp`**: Matches terms using regular expressions.

In [23]:
# 3. Terms Query (The "IN" Operator)
# Matches if host.name is EITHER 'e20666c13238' OR 'unknown_host'
q_terms = {
    "terms": {
        "host.name": ["e20666c13238", "unknown_host"]
    }
}
print("--- TERMS QUERY ---")
search(q_terms)

# 4. IDs Query
# Retrieve specific documents by ID
# (We'll use a dummy ID here, replace with a real one from previous results)
q_ids = {
    "ids": {
        "values": ["e20666c13238"] # Note: This is usually a doc ID, not host name.
    }
}
print("\n--- IDS QUERY ---")
search(q_ids)

# 5. Exists Query (Not Null)
# Finds documents where 'cribl_breaker' field exists.
q_exists = {
    "exists": {
        "field": "cribl_breaker"
    }
}
print("\n--- EXISTS QUERY ---")
search(q_exists)

# 6. Prefix Query
# Finds terms starting with 'e206'
q_prefix = {
    "prefix": {
        "host.name": "e206"
    }
}
print("\n--- PREFIX QUERY ---")
search(q_prefix)

# 7. Regexp Query
# Finds terms matching a regex pattern
q_regexp = {
    "regexp": {
        "host.name": "e206.*"
    }
}
print("\n--- REGEXP QUERY ---")
search(q_regexp)

--- TERMS QUERY ---
⚡ Found 10000 hits in 2ms
--------------------------------------------------
[1] Score: 1.0 | ID: 77dMzJoBDLC7DRstUPNh
    Content: 2025-11-28 21:09:03 UTC [34]: [22841-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[2] Score: 1.0 | ID: 67dMzJoBDLC7DRstSPOR
    Content: 2025-11-28 21:09:00 UTC [34]: [22839-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[3] Score: 1.0 | ID: 7LdMzJoBDLC7DRstSPOR
    Content: 2025-11-28 21:09:02 UTC [39]: [22836-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
------------------------

## 3. Compound Queries (The Glue)

Combine leaf queries using `bool`. This is where you build complex logic.

### The `bool` Query Structure

*   **`must`** (AND): Clause must appear. Contributes to score.
*   **`filter`** (AND): Clause must appear. **No score. Cached.** (Use this for efficiency!)
*   **`should`** (OR): Clause should appear. Increases score. If `must` is present, `should` is optional. If no `must`, at least one `should` is required.
*   **`must_not`** (NOT): Clause must not appear. **No score. Cached.**

In [10]:
# Complex Boolean Query
# Scenario: Find logs that...
# 1. MUST contain 'postgres' (Full text search)
# 2. FILTERED by host 'e20666c13238' (Exact, efficient)
# 3. MUST NOT be from 'test_module' (Exclusion)
# 4. SHOULD contain 'Patroni' (Boost score if present)

q_bool = {
    "bool": {
        "must": [
            { "match": { "_raw": "postgres" } }
        ],
        "filter": [
            { "term": { "host.name": "e20666c13238" } },
            { "range": { "@timestamp": { "gte": "now-24h" } } }
        ],
        "must_not": [
            { "term": { "module": "test_module" } }
        ],
        "should": [
            { "match": { "_raw": "Patroni" } }
        ]
    }
}

print("--- BOOL QUERY ---")
search(q_bool)

--- BOOL QUERY ---
⚡ Found 10000 hits in 38ms
--------------------------------------------------
[1] Score: 0.43351972 | ID: M7ZAypoBDLC7DRsti8d_
    Content: 2025-11-28 11:36:58 UTC [32]: [11-1] user=postgres,db=postgres,app=Patroni heartbeat,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_replication_slot_advance('patroni1', '0/30ABBD0')...
--------------------------------------------------
[2] Score: 0.30072296 | ID: 9bcwzJoBDLC7DRstfeN5
    Content: 2025-11-28 20:38:39 UTC [39]: [21624-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[3] Score: 0.30072296 | ID: 9rcwzJoBDLC7DRstfeN5
    Content: 2025-11-28 20:38:39 UTC [39]: [21625-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
-----

### 3.1 Advanced Compound Queries
Beyond `bool`, OpenSearch offers specialized wrappers to manipulate scores.

*   **`boosting`**: Demote documents without excluding them. (e.g., "Show me apples, but rank 'rotten apples' lower").
*   **`constant_score`**: Ignore relevance and assign a fixed score. Great for performance when you only care about filters but need them in a query context.
*   **`dis_max`**: "Disjunction Max". Useful when searching multiple fields (e.g. title, body) and you want the score to be based on the *best* matching field, not the sum of all matches.
*   **`function_score`**: The ultimate control. Modify scores using math, decay functions (newer is better), or scripts.

In [25]:
# 2. Boosting Query
# Scenario: Find 'postgres' logs, but demote those containing 'error' (maybe we want to see normal logs first?)
# Positive: "postgres" (Score 1.0)
# Negative: "error" (Score * 0.2)
q_boosting = {
    "boosting": {
        "positive": { "match": { "_raw": "postgres" } },
        "negative": { "match": { "_raw": "error" } },
        "negative_boost": 0.2
    }
}
print("--- BOOSTING QUERY ---")
search(q_boosting)

# 3. Constant Score Query
# Wraps a filter query but assigns a score of 1.0 to everything.
# Faster than a standard query because it skips scoring logic.
q_constant = {
    "constant_score": {
        "filter": {
            "term": { "host.name": "e20666c13238" }
        },
        "boost": 1.2
    }
}
print("\n--- CONSTANT SCORE QUERY ---")
search(q_constant)

# 4. Disjunction Max (DisMax)
# Matches documents in 'message' OR '_raw'.
# Score = Score of BEST matching field (not sum).
# tie_breaker: Add 30% of the score from other matching fields.
q_dismax = {
    "dis_max": {
        "queries": [
            { "match": { "message": "failed" } },
            { "match": { "_raw": "failed" } }
        ],
        "tie_breaker": 0.3
    }
}
print("\n--- DIS_MAX QUERY ---")
search(q_dismax)

# 5. Function Score (Decay Example)
# Boost newer documents using a Gaussian decay function.
# "Scale": Score drops to 0.5 if document is 1 day old.
q_func_score = {
    "function_score": {
        "query": { "match_all": {} },
        "functions": [
            {
                "gauss": {
                    "@timestamp": {
                        "origin": "now",
                        "scale": "1d",
                        "decay": 0.5
                    }
                }
            }
        ]
    }
}
print("\n--- FUNCTION SCORE (Decay) ---")
search(q_func_score)

--- BOOSTING QUERY ---
⚡ Found 10000 hits in 74ms
--------------------------------------------------
[1] Score: 0.2251074 | ID: ErFRuZoBDLC7DRstrBYZ
    Content: 2025-11-25 04:42:07 UTC [183]: [2-1] user=postgres,db=postgres,app=Patroni,client=127.0.0.1 LOG:  statement: CHECKPOINT...
--------------------------------------------------
[2] Score: 0.22416529 | ID: g8zctJoBI2mmqWwuFJ95
    Content: 2025-11-24 07:55:12 UTC [56]: [1-1] user=postgres,db=postgres,app=[unknown],client=172.18.0.7 LOG:  statement: SELECT pg_catalog.pg_is_in_recovery()...
--------------------------------------------------
[3] Score: 0.22416529 | ID: xbHEuZoBDLC7DRstMTHZ
    Content: 2025-11-25 06:47:13 UTC [5316]: [1-1] user=postgres,db=postgres,app=[unknown],client=172.18.0.5 LOG:  statement: SELECT pg_catalog.pg_is_in_recovery()...
--------------------------------------------------
[4] Score: 0.22416529 | ID: J7EQupoBDLC7DRstxFLS
    Content: 2025-11-25 08:10:51 UTC [3437]: [1-1] user=postgres,db=postgres,app=[u

## 4. Expensive Queries (Handle with Care)

Some queries are computationally expensive because they scan the inverted index in inefficient ways.

*   **`wildcard`**: `*error*` (Leading wildcards are terrible for performance).
*   **`regexp`**: Regular expressions.
*   **`prefix`**: Starts with...
*   **`fuzzy`**: Approximate matching (Levenshtein distance).

**Efficiency Tip:** Avoid leading wildcards (`*foo`). Use trailing wildcards (`foo*`) if necessary.

In [15]:
# Wildcard Query
# Finds 'postgres', 'postgresql', etc.
# Note: Leading wildcard is slow!
q_wild = {
    "wildcard": {
        "_raw": "*post*"
    }
}
print("--- WILDCARD QUERY ---")
search(q_wild)

# Fuzzy Query
# Finds 'conection' (misspelled) -> 'connection'
# Useful for handling typos but expensive.
q_fuzzy = {
    "fuzzy": {
        "_raw": {
            "value": "postgrs", # Misspelled 'postgres'
            "fuzziness": "AUTO"
        }
    }
}
print("\n--- FUZZY QUERY ---")
search(q_fuzzy)

--- WILDCARD QUERY ---
⚡ Found 10000 hits in 17ms
--------------------------------------------------
[1] Score: 1.0 | ID: 77c3zJoBDLC7DRstSOc7
    Content: 2025-11-28 20:46:04 UTC [39]: [21920-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[2] Score: 1.0 | ID: 8Lc3zJoBDLC7DRstSOc7
    Content: 2025-11-28 20:46:04 UTC [39]: [21921-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[3] Score: 1.0 | ID: 8bc3zJoBDLC7DRstSOc7
    Content: 2025-11-28 20:46:04 UTC [34]: [21924-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------

## 5. 🏎️ The Ultimate Performance Showdown

Let's run a benchmark to see which query types are the most efficient on your current dataset.
We will run each query 50 times and calculate the average execution time.

In [16]:
import time
import statistics

def benchmark(name, query, iterations=50):
    times = []
    # Warmup
    search(query)
    
    for _ in range(iterations):
        # We rely on OpenSearch 'took' for server-side performance
        url = f"{OPENSEARCH_URL}/{INDEX_NAME}/_search"
        try:
            resp = requests.post(url, json={"query": query}, auth=AUTH, verify=VERIFY_SSL)
            if resp.status_code == 200:
                times.append(resp.json()['took'])
        except:
            pass
    
    if not times: return 0
    avg_time = statistics.mean(times)
    return avg_time

# Define the contenders
queries = {
    "Term (Keyword)": {"term": {"host.name": "e20666c13238"}},
    "Range (Date)": {"range": {"@timestamp": {"gte": "now-1h"}}},
    "Bool (Filter)": {
        "bool": {
            "filter": [
                {"term": {"host.name": "e20666c13238"}},
                {"range": {"@timestamp": {"gte": "now-1h"}}}
            ]
        }
    },
    "Match (Text)": {"match": {"_raw": "postgres"}},
    "Multi Match": {"multi_match": {"query": "postgres", "fields": ["_raw", "message"]}},
    "Wildcard (Leading *)": {"wildcard": {"_raw": "*post*"}},
    "Fuzzy": {"fuzzy": {"_raw": {"value": "postgrs", "fuzziness": "AUTO"}}}
}

print(f"{'Query Type':<25} | {'Avg Time (ms)':<15} | {'Efficiency Rating'}")
print("-" * 60)

results = []
for name, q in queries.items():
    avg = benchmark(name, q)
    results.append((name, avg))

# Sort by speed
results.sort(key=lambda x: x[1])

for name, avg in results:
    rating = "🚀 Instant" if avg < 1 else "⚡ Fast" if avg < 5 else "🐢 Slow" if avg < 20 else "🐌 Terrible"
    print(f"{name:<25} | {avg:<15.2f} | {rating}")

Query Type                | Avg Time (ms)   | Efficiency Rating
------------------------------------------------------------
⚡ Found 10000 hits in 1ms
--------------------------------------------------
[1] Score: 1.0065804 | ID: Kbc3zJoBDLC7DRstrejS
    Content: 2025-11-28 20:46:31 UTC [39]: [21938-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[2] Score: 1.0065804 | ID: Krc3zJoBDLC7DRstrejS
    Content: 2025-11-28 20:46:31 UTC [39]: [21939-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[3] Score: 1.0065804 | ID: K7c3zJoBDLC7DRstrejS
    Content: 2025-11-28 20:46:31 UTC [34]: [21942-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e

In [17]:
# Print results for the agent to read
print(results)

[('Term (Keyword)', 0.02), ('Range (Date)', 0.04), ('Bool (Filter)', 0.18), ('Match (Text)', 0.2), ('Multi Match', 0.66), ('Fuzzy', 1.1), ('Wildcard (Leading *)', 4)]


## 6. Joining Queries (Nested & Parent-Child)

OpenSearch is distributed, so standard SQL JOINs are expensive. Instead, it offers two specific ways to handle relationships:

1.  **`nested`**: For arrays of objects (e.g., a blog post with multiple comments). Each object is indexed as a hidden separate document.
2.  **`join` field**: For parent-child relationships between documents (e.g., Companies and Employees).

**⚠️ Note:** These require specific mappings. We will create temporary indices to demonstrate them.

> **🚨 CRITICAL WARNING:**
> If `search.allow_expensive_queries` is set to `false` in your cluster settings, **joining queries will not be executed**.
> This is a common reason for unexpected failures in production environments where strict performance controls are in place.

In [26]:
# --- DEMO: NESTED QUERIES ---

# 1. Setup: Create an index with NESTED mapping
nested_index = "demo_nested"
mapping = {
    "mappings": {
        "properties": {
            "product": { "type": "text" },
            "reviews": {
                "type": "nested",  # <--- The Magic Keyword
                "properties": {
                    "user": { "type": "keyword" },
                    "stars": { "type": "integer" },
                    "comment": { "type": "text" }
                }
            }
        }
    }
}

# Delete if exists, then create
requests.delete(f"{OPENSEARCH_URL}/{nested_index}", auth=AUTH, verify=VERIFY_SSL)
requests.put(f"{OPENSEARCH_URL}/{nested_index}", json=mapping, auth=AUTH, verify=VERIFY_SSL)

# 2. Index Data
doc = {
    "product": "Super Widget",
    "reviews": [
        { "user": "alice", "stars": 5, "comment": "Loved it!" },
        { "user": "bob", "stars": 1, "comment": "Terrible." }
    ]
}
requests.post(f"{OPENSEARCH_URL}/{nested_index}/_doc/1?refresh=true", json=doc, auth=AUTH, verify=VERIFY_SSL)

# 3. The Problem: Standard Query Fails on Nested Objects
# If we try to find a review with user="alice" AND stars=1 (which doesn't exist in a single object),
# a standard object array would match (because it flattens the arrays).
# But 'nested' keeps them separate.

# 4. The Solution: Nested Query
q_nested = {
    "nested": {
        "path": "reviews",
        "query": {
            "bool": {
                "must": [
                    { "term": { "reviews.user": "alice" } },
                    { "term": { "reviews.stars": 5 } }  # Alice gave 5 stars, so this matches
                ]
            }
        }
    }
}

print("--- NESTED QUERY (Alice + 5 Stars) ---")
search(q_nested, index=nested_index)

# Verify mismatch (Alice + 1 Star should fail)
q_nested_fail = {
    "nested": {
        "path": "reviews",
        "query": {
            "bool": {
                "must": [
                    { "term": { "reviews.user": "alice" } },
                    { "term": { "reviews.stars": 1 } }
                ]
            }
        }
    }
}
print("\n--- NESTED QUERY (Alice + 1 Star -> Should Fail) ---")
search(q_nested_fail, index=nested_index)

--- NESTED QUERY (Alice + 5 Stars) ---
⚡ Found 1 hits in 11ms
--------------------------------------------------
[1] Score: 2.0 | ID: 1
    Content: {'product': 'Super Widget', 'reviews': [{'user': 'alice', 'stars': 5, 'comment': 'Loved it!'}, {'use...
--------------------------------------------------

--- NESTED QUERY (Alice + 1 Star -> Should Fail) ---
⚡ Found 0 hits in 0ms
--------------------------------------------------


In [27]:
# --- DEMO: PARENT-CHILD (JOIN) QUERIES ---

# 1. Setup: Create index with JOIN field
join_index = "demo_joins"
mapping_join = {
    "mappings": {
        "properties": {
            "name": { "type": "text" },
            "my_join_field": { 
                "type": "join",
                "relations": {
                    "company": "employee"  # Parent: company, Child: employee
                }
            }
        }
    }
}

requests.delete(f"{OPENSEARCH_URL}/{join_index}", auth=AUTH, verify=VERIFY_SSL)
requests.put(f"{OPENSEARCH_URL}/{join_index}", json=mapping_join, auth=AUTH, verify=VERIFY_SSL)

# 2. Index Parent (Company)
parent_doc = {
    "name": "TechCorp",
    "my_join_field": "company"
}
requests.put(f"{OPENSEARCH_URL}/{join_index}/_doc/1?refresh=true", json=parent_doc, auth=AUTH, verify=VERIFY_SSL)

# 3. Index Child (Employee)
# CRITICAL: Must specify 'routing' (parent ID) to ensure they live on the same shard.
child_doc = {
    "name": "John Doe",
    "my_join_field": {
        "name": "employee",
        "parent": "1"  # ID of the parent company
    }
}
requests.put(f"{OPENSEARCH_URL}/{join_index}/_doc/2?routing=1&refresh=true", json=child_doc, auth=AUTH, verify=VERIFY_SSL)

# 4. Has Child Query
# Find companies that have an employee named "John"
q_has_child = {
    "has_child": {
        "type": "employee",
        "query": {
            "match": { "name": "John" }
        }
    }
}
print("--- HAS CHILD QUERY (Find Parent via Child) ---")
search(q_has_child, index=join_index)

# 5. Has Parent Query
# Find employees who work for "TechCorp"
q_has_parent = {
    "has_parent": {
        "parent_type": "company",
        "query": {
            "match": { "name": "TechCorp" }
        }
    }
}
print("\n--- HAS PARENT QUERY (Find Child via Parent) ---")
search(q_has_parent, index=join_index)

# 6. Parent ID Query
# Find all children of a specific parent ID
q_parent_id = {
    "parent_id": {
        "type": "employee",
        "id": "1"
    }
}
print("\n--- PARENT ID QUERY (Direct Lookup) ---")
search(q_parent_id, index=join_index)

--- HAS CHILD QUERY (Find Parent via Child) ---
⚡ Found 1 hits in 15ms
--------------------------------------------------
[1] Score: 1.0 | ID: 1
    Content: {'name': 'TechCorp', 'my_join_field': 'company'}...
--------------------------------------------------

--- HAS PARENT QUERY (Find Child via Parent) ---
⚡ Found 1 hits in 0ms
--------------------------------------------------
[1] Score: 1.0 | ID: 2
    Content: {'name': 'John Doe', 'my_join_field': {'name': 'employee', 'parent': '1'}}...
--------------------------------------------------

--- PARENT ID QUERY (Direct Lookup) ---
⚡ Found 1 hits in 0ms
--------------------------------------------------
[1] Score: 0.082873434 | ID: 2
    Content: {'name': 'John Doe', 'my_join_field': {'name': 'employee', 'parent': '1'}}...
--------------------------------------------------


## 7. Span Queries (Positional Search)

Span queries are low-level, positional queries. They are primarily used for legal search, patent search, or anywhere where the **exact distance** and **order** of terms matters significantly.

Unlike `match_phrase` (which uses a simple `slop`), Span queries allow complex logic like:
*   "Find 'lawsuit' within 5 words of 'patent', but NOT if 'dismissed' is between them."
*   "Find 'Chapter 1' only if it appears in the first 10 tokens of the document."

**Key Types:**
*   **`span_term`**: The atom. Matches a single term.
*   **`span_near`**: Matches spans near each other.
*   **`span_first`**: Matches near the beginning of the field.
*   **`span_not`**: Excludes matches that overlap with another span.
*   **`span_or`**: Combines multiple spans.

In [29]:
# --- DEMO: SPAN QUERIES ---

# 1. Setup: Create Index
span_index = "demo_span"
# We use standard analyzer so "Long-sleeved" -> ["long", "sleeved"]
requests.delete(f"{OPENSEARCH_URL}/{span_index}", auth=AUTH, verify=VERIFY_SSL)
requests.put(f"{OPENSEARCH_URL}/{span_index}", json={}, auth=AUTH, verify=VERIFY_SSL)

# 2. Index Data (Clothing Descriptions)
docs = [
    { "description": "Long-sleeved dress shirt with a formal collar and button cuffs." },
    { "description": "Beautiful long dress in red silk, perfect for formal events." },
    { "description": "Short-sleeved shirt with a button-down collar." },
    { "description": "A set of two midi silk shirt dresses with long sleeves in black." }
]

for i, d in enumerate(docs):
    requests.post(f"{OPENSEARCH_URL}/{span_index}/_doc/{i+1}?refresh=true", json=d, auth=AUTH, verify=VERIFY_SSL)

# 3. Span Term (The Building Block)
# Matches "dress" exactly.
q_span_term = {
    "span_term": { "description": "dress" }
}
print("--- SPAN TERM ('dress') ---")
search(q_span_term, index=span_index)

# 4. Span Near (Proximity)
# Find "dress" and "shirt" within 5 words of each other, in order.
q_span_near = {
    "span_near": {
        "clauses": [
            { "span_term": { "description": "dress" } },
            { "span_term": { "description": "shirt" } }
        ],
        "slop": 5,
        "in_order": True
    }
}
print("\n--- SPAN NEAR ('dress' near 'shirt') ---")
search(q_span_near, index=span_index)

# 5. Span First (Beginning of Field)
# Find "long" only if it appears in the first 3 words.
q_span_first = {
    "span_first": {
        "match": {
            "span_term": { "description": "long" }
        },
        "end": 3
    }
}
print("\n--- SPAN FIRST ('long' in first 3 words) ---")
search(q_span_first, index=span_index)

# 6. Span Not (Exclusion)
# Find "dress" and "shirt" near each other...
# BUT exclude if "silk" is between them.
q_span_not = {
    "span_not": {
        "include": {
            "span_near": {
                "clauses": [
                    { "span_term": { "description": "dress" } },
                    { "span_term": { "description": "shirt" } }
                ],
                "slop": 10,
                "in_order": False
            }
        },
        "exclude": {
            "span_term": { "description": "silk" }
        }
    }
}
print("\n--- SPAN NOT ('dress' near 'shirt', NO 'silk') ---")
search(q_span_not, index=span_index)

--- SPAN TERM ('dress') ---
⚡ Found 2 hits in 0ms
--------------------------------------------------
[1] Score: 0.3213265 | ID: 2
    Content: {'description': 'Beautiful long dress in red silk, perfect for formal events.'}...
--------------------------------------------------
[2] Score: 0.3090465 | ID: 1
    Content: {'description': 'Long-sleeved dress shirt with a formal collar and button cuffs.'}...
--------------------------------------------------

--- SPAN NEAR ('dress' near 'shirt') ---
⚡ Found 1 hits in 1ms
--------------------------------------------------
[1] Score: 0.4680735 | ID: 1
    Content: {'description': 'Long-sleeved dress shirt with a formal collar and button cuffs.'}...
--------------------------------------------------

--- SPAN FIRST ('long' in first 3 words) ---
⚡ Found 2 hits in 5ms
--------------------------------------------------
[1] Score: 0.165346 | ID: 2
    Content: {'description': 'Beautiful long dress in red silk, perfect for formal events.'}...
-------

## 8. Match All Query (The "Select *")

The `match_all` query is the simplest query. It matches **all** documents.
It is the default query if no query is specified.

**Parameters:**
*   **`boost`**: (Optional) Floating point. Sets the relevance score for all documents. Default is 1.0.
*   **`_name`**: (Optional) String. Used for tagging queries (useful when debugging complex queries).

**Counterpart:**
*   **`match_none`**: Matches **no** documents. Rarely used, but exists.

In [30]:
# --- DEMO: MATCH ALL ---

# 1. Basic Match All
# Matches everything. Score is 1.0.
q_all = {
    "match_all": {}
}
print("--- MATCH ALL (Default) ---")
search(q_all)

# 2. Match All with Boost
# Matches everything. Score is 2.5.
q_all_boost = {
    "match_all": {
        "boost": 2.5
    }
}
print("\n--- MATCH ALL (Boost 2.5) ---")
search(q_all_boost)

# 3. Match None
# Matches nothing.
q_none = {
    "match_none": {}
}
print("\n--- MATCH NONE ---")
search(q_none)

--- MATCH ALL (Default) ---
⚡ Found 10000 hits in 1ms
--------------------------------------------------
[1] Score: 1.0 | ID: e7dYzJoBDLC7DRstP_vO
    Content: 2025-11-28 21:22:05 UTC [34]: [23361-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[2] Score: 1.0 | ID: eLdYzJoBDLC7DRstN_v-
    Content: 2025-11-28 21:22:04 UTC [34]: [23360-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[3] Score: 1.0 | ID: ebdYzJoBDLC7DRstN_v-
    Content: 2025-11-28 21:22:04 UTC [39]: [23356-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
----------------

## 9. Specialized Queries

These queries handle specific, often complex use cases beyond standard text or term matching.

### Key Specialized Queries

1.  **`more_like_this` (MLT)**:
    *   **Concept**: "Find me other documents that look like this text." It analyzes the input text, extracts key terms (using tf-idf), and builds a query behind the scenes.
    *   **Use Case**: "Related Articles", "Similar Products".
    *   **Limitation**: Can be slow on large fields; requires tuning `min_term_freq` and `min_doc_freq` to avoid noise.

2.  **`script`**:
    *   **Concept**: Use a custom script (Painless language) to filter documents.
    *   **Use Case**: Complex logic not supported by DSL (e.g., "doc['field_a'].value > doc['field_b'].value").
    *   **Limitation**: **Performance Killer**. Scripts are compiled and executed per document. Use only when absolutely necessary.

3.  **`script_score`**:
    *   **Concept**: Customize the relevance score using a script.
    *   **Use Case**: "Score = popularity * 0.5 + relevance".
    *   **Limitation**: Slower than standard scoring.

4.  **`distance_feature`**:
    *   **Concept**: Boosts score based on proximity to a specific date or geo-point.
    *   **Use Case**: "Boost news articles closer to today" or "Boost restaurants closer to my location".
    *   **Limitation**: Only works on `date`, `date_nanos`, or `geo_point` fields.

5.  **`percolate`**:
    *   **Concept**: "Reverse Search". You index *queries* and then "search" with a *document* to see which queries match it.
    *   **Use Case**: Alerting systems (e.g., "Notify me if a new log matches 'error' AND 'production'").
    *   **Limitation**: Requires a specific `percolator` field type.

6.  **`knn` (k-Nearest Neighbors)**:
    *   **Concept**: Vector search. Finds documents with similar vector embeddings.
    *   **Use Case**: Semantic search, image search, recommendation engines.
    *   **Limitation**: Requires `knn` plugin and specific index settings (HNSW graphs).

7.  **`rank_feature`**:
    *   **Concept**: Optimized way to boost scores based on numeric features (like PageRank or URL length).
    *   **Limitation**: Field must be mapped as `rank_feature`.

In [31]:
# --- DEMO: SPECIALIZED QUERIES ---

# 1. More Like This (MLT)
# Find documents similar to the text "dress shirt"
# It will extract "dress" and "shirt" as interesting terms.
q_mlt = {
    "more_like_this": {
        "fields": ["description"],
        "like": "dress shirt",
        "min_term_freq": 1,
        "min_doc_freq": 1
    }
}
print("--- MORE LIKE THIS ('dress shirt') ---")
search(q_mlt, index=span_index) # Using the clothing index from Span demo

# 2. Script Query (Filter)
# Find documents where the description length is greater than 50 characters.
# Note: 'doc' access is fast, '_source' is slow.
q_script = {
    "script": {
        "script": {
            "source": "doc['description.keyword'].value.length() > 50",
            "lang": "painless"
        }
    }
}
# Note: This might fail if 'description' doesn't have a keyword field or doc values enabled.
# Let's try a safer one on the main index using a numeric field if available, or just skip if no suitable field.
# We'll try on the 'patronidata' index assuming there's some field we can check.
# Actually, let's use the span_index and check description length assuming it has a keyword subfield (default in many setups).
# If it fails, it illustrates the limitation!
print("\n--- SCRIPT QUERY (Length > 50) ---")
# We need to ensure mapping allows this. The span_index was created with default mapping? 
# In the span demo we did: requests.put(..., json={}) -> Default dynamic mapping.
# Default dynamic mapping for string is "text" with "keyword" subfield.
try:
    search(q_script, index=span_index)
except:
    print("Script query failed (likely missing doc values or mapping issue).")

# 3. Script Score
# Custom scoring: Score = 1.0 + (length of description * 0.1)
q_script_score = {
    "function_score": {
        "query": { "match_all": {} },
        "script_score": {
            "script": {
                "source": "1.0 + (doc['description.keyword'].value.length() * 0.1)"
            }
        }
    }
}
print("\n--- SCRIPT SCORE ---")
try:
    search(q_script_score, index=span_index)
except:
    print("Script score failed.")

# 4. Distance Feature
# Boost documents closer to "now".
# Note: Requires a date field. We'll use the main 'patronidata' index which has @timestamp.
q_distance = {
    "bool": {
        "must": {"match_all": {}},
        "should": {
            "distance_feature": {
                "field": "@timestamp",
                "pivot": "7d", # Score halves every 7 days distance
                "origin": "now",
                "boost": 2.0
            }
        }
    }
}
print("\n--- DISTANCE FEATURE (Boost recent logs) ---")
search(q_distance, index=INDEX_NAME)

--- MORE LIKE THIS ('dress shirt') ---
⚡ Found 4 hits in 14ms
--------------------------------------------------
[1] Score: 0.46807355 | ID: 1
    Content: {'description': 'Long-sleeved dress shirt with a formal collar and button cuffs.'}...
--------------------------------------------------
[2] Score: 0.3213265 | ID: 2
    Content: {'description': 'Beautiful long dress in red silk, perfect for formal events.'}...
--------------------------------------------------
[3] Score: 0.17962044 | ID: 3
    Content: {'description': 'Short-sleeved shirt with a button-down collar.'}...
--------------------------------------------------
[4] Score: 0.14773516 | ID: 4
    Content: {'description': 'A set of two midi silk shirt dresses with long sleeves in black.'}...
--------------------------------------------------

--- SCRIPT QUERY (Length > 50) ---
⚡ Found 3 hits in 43ms
--------------------------------------------------
[1] Score: 1.0 | ID: 1
    Content: {'description': 'Long-sleeved dress shirt

## 10. Percolate Query (Reverse Search)

The `percolate` query is a "reverse search". Instead of indexing documents and searching them with queries, you **index queries** and then "search" them with a document.

**Use Case:** Alerting.
*   You store queries like "error AND production" or "disk space > 90%".
*   When a new log arrives, you "percolate" it to see which stored queries it matches.
*   If it matches, you trigger an alert.

**Steps:**
1.  Create an index with a `percolator` field type.
2.  Index your queries into this field.
3.  Run a `percolate` query with a document to see which queries match.

In [32]:
# --- DEMO: PERCOLATE QUERY ---

# 1. Setup: Create 'alerts' index with a percolator field
alerts_index = "alerts"
mapping_percolate = {
    "mappings": {
        "properties": {
            "query":   { "type": "percolator" },
            "tag":     { "type": "keyword" },
            "message": { "type": "text" } # The field our queries will target
        }
    }
}

requests.delete(f"{OPENSEARCH_URL}/{alerts_index}", auth=AUTH, verify=VERIFY_SSL)
requests.put(f"{OPENSEARCH_URL}/{alerts_index}", json=mapping_percolate, auth=AUTH, verify=VERIFY_SSL)

# 2. Register Stored Queries
# We index QUERIES as documents.
stored_queries = [
    {
        "id": "1",
        "body": {
            "tag": "error",
            "query": { "match": { "message": "error" } }
        }
    },
    {
        "id": "2",
        "body": {
            "tag": "warn",
            "query": { "match_phrase": { "message": "disk space" } }
        }
    },
    {
        "id": "3",
        "body": {
            "tag": "urgent",
            "query": {
                "bool": {
                    "must":   { "match": { "message": "failure" } },
                    "filter": { "term":  { "tag": "error" } }
                }
            }
        }
    }
]

print("--- Registering Queries ---")
for sq in stored_queries:
    url = f"{OPENSEARCH_URL}/{alerts_index}/_doc/{sq['id']}?refresh=true"
    requests.post(url, json=sq['body'], auth=AUTH, verify=VERIFY_SSL)
    print(f"Registered query {sq['id']} (Tag: {sq['body']['tag']})")

# 3. Percolate a Document (Inline)
# "Here is a document. Which queries match it?"
doc_inline = {
    "message": "disk space failure on node 7",
    "tag": "error"
}

q_percolate_inline = {
    "percolate": {
        "field": "query",
        "document": doc_inline
    }
}

print("\n--- PERCOLATE INLINE (Check 'disk space failure...') ---")
# We highlight to see WHY it matched
search(q_percolate_inline, index=alerts_index)

# 4. Percolate Multiple Documents
# Check multiple docs at once.
q_percolate_multi = {
    "percolate": {
        "field": "query",
        "documents": [
            { "message": "fatal error reading disk", "tag": "error" },
            { "message": "high cpu usage" }
        ]
    }
}
print("\n--- PERCOLATE MULTIPLE ---")
search(q_percolate_multi, index=alerts_index)

# 5. Percolate using a Stored Source Document
# First, store a real document in a separate index 'docs'
docs_index = "docs"
mapping_docs = { "mappings": { "properties": { "message": { "type": "text" }, "tag": { "type": "keyword" } } } }
requests.delete(f"{OPENSEARCH_URL}/{docs_index}", auth=AUTH, verify=VERIFY_SSL)
requests.put(f"{OPENSEARCH_URL}/{docs_index}", json=mapping_docs, auth=AUTH, verify=VERIFY_SSL)

# Index the document we want to check
requests.post(f"{OPENSEARCH_URL}/{docs_index}/_doc/42?refresh=true", json={
    "message": "disk space is critically low",
    "tag": "warn"
}, auth=AUTH, verify=VERIFY_SSL)

# Now percolate by referencing that stored document
q_percolate_stored = {
    "percolate": {
        "field": "query",
        "index": docs_index,
        "id": "42",
        "version": 1 # Optional, but good practice
    }
}
print("\n--- PERCOLATE STORED DOC (ID: 42) ---")
search(q_percolate_stored, index=alerts_index)

--- Registering Queries ---
Registered query 1 (Tag: error)
Registered query 2 (Tag: warn)
Registered query 3 (Tag: urgent)

--- PERCOLATE INLINE (Check 'disk space failure...') ---
⚡ Found 2 hits in 27ms
--------------------------------------------------
[1] Score: 0.26152915 | ID: 2
    Content: {'tag': 'warn', 'query': {'match_phrase': {'message': 'disk space'}}}...
--------------------------------------------------
[2] Score: 0.13076457 | ID: 3
    Content: {'tag': 'urgent', 'query': {'bool': {'must': {'match': {'message': 'failure'}}, 'filter': {'term': {...
--------------------------------------------------

--- PERCOLATE MULTIPLE ---
⚡ Found 1 hits in 20ms
--------------------------------------------------
[1] Score: 0.29767057 | ID: 1
    Content: {'tag': 'error', 'query': {'match': {'message': 'error'}}}...
--------------------------------------------------

--- PERCOLATE STORED DOC (ID: 42) ---
⚡ Found 1 hits in 4ms
--------------------------------------------------
[1] Score

## 11. Minimum Should Match (Precision Control)

The `minimum_should_match` parameter gives you precise control over how many optional clauses (`should` in bool, or terms in `match`) must match for a document to be returned.

**Valid Values:**

| Value | Example | Description |
| :--- | :--- | :--- |
| **Integer** | `2` | Must match at least 2 clauses. |
| **Negative Integer** | `-1` | Must match (Total Clauses - 1). |
| **Percentage** | `75%` | Must match 75% of clauses (rounded down). |
| **Negative Percentage** | `-25%` | Allowed to miss 25% of clauses. |
| **Combination** | `2<75%` | If clauses <= 2, match all. If > 2, match 75%. |

**Defaults:**
*   If `bool` query has `must` or `filter`: Default is **0** (optional).
*   If `bool` query has ONLY `should`: Default is **1** (at least one must match).

In [33]:
# --- DEMO: MINIMUM SHOULD MATCH ---

# 1. Match Query with Minimum Should Match
# "postgres error failed fatal" -> 4 terms.
# "3" means at least 3 of these terms must be present.
q_min_match = {
    "match": {
        "message": {
            "query": "postgres error failed fatal",
            "minimum_should_match": "3"
        }
    }
}
print("--- MATCH QUERY (Min 3 terms) ---")
# We'll run this against the main index, though we might not have a doc with 3 of these.
# It illustrates the syntax.
search(q_min_match)

# 2. Bool Query with Percentages
# We have 4 should clauses.
# "50%" of 4 is 2. So at least 2 clauses must match.
q_bool_min = {
    "bool": {
        "should": [
            { "term": { "host.name": "e20666c13238" } }, # Match
            { "match": { "_raw": "postgres" } },         # Match
            { "term": { "tag": "impossible_tag" } },     # No Match
            { "term": { "status": "404" } }              # No Match
        ],
        "minimum_should_match": "50%"
    }
}
print("\n--- BOOL QUERY (Min 50% of 4 clauses) ---")
search(q_bool_min)

# 3. Combination Logic (The "High Availability" Logic)
# "2<-1":
#   If <= 2 clauses: Match all.
#   If > 2 clauses: Match all but 1.
q_combo = {
    "bool": {
        "should": [
            { "match": { "_raw": "postgres" } },
            { "match": { "_raw": "error" } },
            { "match": { "_raw": "fatal" } }
        ],
        "minimum_should_match": "2<-1" 
    }
}
# Here we have 3 clauses (> 2). So we need (3 - 1) = 2 matches.
print("\n--- COMBINATION (2<-1) ---")
search(q_combo)

--- MATCH QUERY (Min 3 terms) ---
⚡ Found 0 hits in 2ms
--------------------------------------------------

--- BOOL QUERY (Min 50% of 4 clauses) ---
⚡ Found 10000 hits in 24ms
--------------------------------------------------
[1] Score: 1.19512 | ID: G7ZAypoBDLC7DRstg8e3
    Content: 2025-11-28 11:36:49 UTC [60]: [1-1] user=postgres,db=postgres,app=[unknown],client=172.18.0.5, e=00000 LOG:  statement: SELECT pg_catalog.pg_is_in_recovery()...
--------------------------------------------------
[2] Score: 1.191502 | ID: M7ZAypoBDLC7DRsti8d_
    Content: 2025-11-28 11:36:58 UTC [32]: [11-1] user=postgres,db=postgres,app=Patroni heartbeat,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_replication_slot_advance('patroni1', '0/30ABBD0')...
--------------------------------------------------
[3] Score: 1.1358372 | ID: 77hizJoBDLC7DRst7gH4
    Content: 2025-11-28 21:33:45 UTC [34]: [23826-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement

## 12. Rewrite Parameter (Performance vs. Accuracy)

Multi-term queries like `wildcard`, `prefix`, `fuzzy`, and `regexp` don't just run as-is. OpenSearch **rewrites** them into a set of simpler queries (usually a giant `bool` query with many `term` clauses).

The `rewrite` parameter controls this behavior.

**Why care?**
1.  **Performance**: Calculating scores for 1000 expanded terms is slow.
2.  **Limits**: The default limit for boolean clauses is 1024. A `wildcard: "a*"` could easily exceed this and fail.
3.  **Scoring**: Do you need relevance (TF/IDF) for every matching term, or just "yes/no"?

**Common Modes:**
*   **`constant_score`** (Default): Wraps everything in a filter. Fast. Score is always 1.0.
*   **`scoring_boolean`**: Calculates relevance for every term. Most accurate, but expensive. Can hit clause limits.
*   **`top_terms_N`** (e.g., `top_terms_10`): Only calculates scores for the N most frequent terms. Good compromise.

In [34]:
# --- DEMO: REWRITE PARAMETER ---

# 1. Constant Score (Default)
# Fast, no scoring overhead. Score is 1.0.
q_rewrite_const = {
    "wildcard": {
        "_raw": {
            "value": "post*",
            "rewrite": "constant_score"
        }
    }
}
print("--- REWRITE: CONSTANT SCORE (Default) ---")
search(q_rewrite_const)

# 2. Scoring Boolean
# Calculates TF/IDF for every expanded term (postgres, postgresql, posting, etc.)
# Slower, but gives better relevance.
q_rewrite_score = {
    "wildcard": {
        "_raw": {
            "value": "post*",
            "rewrite": "scoring_boolean"
        }
    }
}
print("\n--- REWRITE: SCORING BOOLEAN (Full Relevance) ---")
search(q_rewrite_score)

# 3. Top Terms N
# Only scores the top 2 most frequent terms matching "post*".
# Ignores the rest for scoring purposes.
q_rewrite_top = {
    "wildcard": {
        "_raw": {
            "value": "post*",
            "rewrite": "top_terms_2"
        }
    }
}
print("\n--- REWRITE: TOP TERMS 2 (Optimization) ---")
search(q_rewrite_top)

--- REWRITE: CONSTANT SCORE (Default) ---
⚡ Found 10000 hits in 14ms
--------------------------------------------------
[1] Score: 1.0 | ID: XrhjzJoBDLC7DRstugIl
    Content: 2025-11-28 21:34:37 UTC [34]: [23861-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[2] Score: 1.0 | ID: X7hjzJoBDLC7DRstugIl
    Content: 2025-11-28 21:34:38 UTC [33]: [3591-1] user=postgres,db=postgres,app=Patroni heartbeat,client=127.0.0.1, e=00000 LOG:  statement: SELECT CASE WHEN pg_catalog.pg_is_in_recovery() THEN 0 ELSE ('x' || pg...
--------------------------------------------------
[3] Score: 1.0 | ID: YbhjzJoBDLC7DRstugIl
    Content: 2025-11-28 21:34:39 UTC [32]: [3597-1] user=postgres,db=postgres,app=Patroni heartbeat,client=127.0.0.1, e=00000 LOG:  statement: SELECT CASE WHEN pg_catalog.pg_is_in_recovery() THEN 0 ELSE ('x' || pg...
-

## 13. Rescore (The Second Pass)

Sometimes, running a complex, expensive query (like a `match_phrase` with a large slop, or a heavy script) on **all** documents is too slow.

**The Solution:** `rescore`.

It works in two passes:
1.  **First Pass**: Run a cheap, fast query (e.g., `match`) on the whole index to get the top N results (e.g., top 100).
2.  **Second Pass**: Run the expensive query **only** on those top N results to re-rank them.

**Parameters:**
*   **`window_size`**: How many documents to rescore (default 10).
*   **`query_weight`**: Weight of the original query's score.
*   **`rescore_query_weight`**: Weight of the second pass score.
*   **`score_mode`**: How to combine scores (`total`, `multiply`, `avg`, `max`, `min`).

**Use Case:**
*   "Find all docs with 'OpenSearch' (Cheap), then boost the ones where 'OpenSearch' is near 'Optimization' (Expensive Phrase)."
*   "Find all products (Cheap), then boost by a complex popularity script (Expensive)."

In [37]:
# --- DEMO: RESCORE ---

# 1. The Baseline (Cheap Query)
# Just find documents with "postgres" or "error".
# The ranking is purely based on TF/IDF of these single terms.
q_baseline = {
    "match": {
        "message": "postgres error"
    }
}
print("--- BASELINE QUERY (No Rescore) ---")
search(q_baseline)

# 2. Rescore with Phrase Match
# Strategy:
#   1. Find top 50 docs with "postgres" OR "error" (Cheap).
#   2. Re-rank those 50 by checking if they contain the exact phrase "postgres error" (Expensive).
body_rescore_phrase = {
    "query": {
        "match": {
            "message": "postgres error"
        }
    },
    "rescore": {
        "window_size": 50,
        "query": {
            "rescore_query": {
                "match_phrase": {
                    "message": {
                        "query": "postgres error",
                        "slop": 0
                    }
                }
            },
            "query_weight": 1.0,
            "rescore_query_weight": 2.0  # Give the phrase match double importance
        }
    }
}
print("\n--- RESCORE: PHRASE MATCH (Boost Exact Phrase) ---")
search(body=body_rescore_phrase)

# 3. Rescore with Script (The Heavy Lifter)
# Strategy:
#   1. Find top 50 docs.
#   2. Re-rank using a script (e.g., boost based on length of message).
#   Note: Running this script on 1M docs would be slow. Running on 50 is fast.
body_rescore_script = {
    "query": {
        "match": {
            "message": "postgres"
        }
    },
    "rescore": {
        "window_size": 50,
        "query": {
            "score_mode": "multiply", # Multiply original score by script result
            "rescore_query": {
                "function_score": {
                    "script_score": {
                        "script": {
                            "source": "Math.log10(doc['message.keyword'].value.length() + 1)"
                        }
                    }
                }
            }
        }
    }
}
print("\n--- RESCORE: SCRIPT (Boost Long Messages) ---")
# Note: This requires 'message.keyword' to exist. If not, it might fail or return 0 hits if caught.
try:
    search(body=body_rescore_script)
except:
    print("Script rescore failed (likely mapping issue).")

--- BASELINE QUERY (No Rescore) ---
⚡ Found 1 hits in 1ms
--------------------------------------------------
[1] Score: 0.13076457 | ID: mrTOxJoBDLC7DRst4JdI
    Content: CRITICAL error: This is a test log to trigger the YAML monitor...
--------------------------------------------------

--- RESCORE: PHRASE MATCH (Boost Exact Phrase) ---
⚡ Found 1 hits in 6ms
--------------------------------------------------
[1] Score: 0.13076457 | ID: mrTOxJoBDLC7DRst4JdI
    Content: CRITICAL error: This is a test log to trigger the YAML monitor...
--------------------------------------------------

--- RESCORE: SCRIPT (Boost Long Messages) ---
⚡ Found 0 hits in 27ms
--------------------------------------------------


## 14. Regex Syntax (Power & Peril)

OpenSearch supports Regular Expressions via the `regexp` query.
**Crucial Note:** It uses the **Apache Lucene** regex engine, NOT standard PCRE (Perl Compatible Regular Expressions).

### Key Differences from Standard Regex
1.  **Anchors are Implicit:** You cannot use `^` (start) or `$` (end). The pattern **must match the entire token** by default.
    *   `error` matches "error".
    *   `error` DOES NOT match "error_log" (unless you use `error.*`).
2.  **No Lookarounds:** Lookahead `(?=...)` and lookbehind `(?<=...)` are NOT supported.
3.  **Performance:** Regex queries are **term-level** queries. They scan the inverted index for *every term* that matches the pattern. `.*error.*` is a performance disaster.

### Supported Operators
*   `.`: Any character.
*   `?`: 0 or 1.
*   `+`: 1 or more.
*   `*`: 0 or more.
*   `{n,m}`: Repetitions (e.g., `{3,5}`).
*   `|`: OR.
*   `[]`: Character classes (e.g., `[a-z]`).
*   `~`: Complement (requires `COMPLEMENT` flag).
*   `&`: Intersection (requires `INTERSECTION` flag).
*   `<1-100>`: Numeric range (requires `INTERVAL` flag).

In [38]:
# --- DEMO: REGEX SYNTAX ---

# 1. Basic Regex
# Find terms starting with 'post' and ending with 's' or 'l'
# Matches: "postgres", "postgresql"
q_regex_basic = {
    "regexp": {
        "_raw": "post.*[sl]"
    }
}
print("--- REGEX: BASIC (post.*[sl]) ---")
search(q_regex_basic)

# 2. Numeric Range (Interval Flag)
# Find "error" followed by a number between 400 and 599.
# Syntax: <min-max>
# Requires flags: "INTERVAL"
q_regex_interval = {
    "regexp": {
        "message": {
            "value": "error<400-599>",
            "flags": "INTERVAL"
        }
    }
}
print("\n--- REGEX: INTERVAL (error<400-599>) ---")
# We'll try this, but we might not have matching data.
# It illustrates the syntax.
search(q_regex_interval)

# 3. Intersection (AND Operator)
# Find terms that match TWO patterns simultaneously.
# Example: Must contain "post" AND be exactly 8 characters long.
# Syntax: pattern1&pattern2
# Requires flags: "INTERSECTION"
# Pattern 1: .*post.* (Contains 'post')
# Pattern 2: .{8} (Exactly 8 chars)
q_regex_intersection = {
    "regexp": {
        "_raw": {
            "value": ".*post.*&.{8}",
            "flags": "INTERSECTION"
        }
    }
}
print("\n--- REGEX: INTERSECTION (Contains 'post' AND Length=8) ---")
search(q_regex_intersection)

# 4. Complement (NOT Operator)
# Find terms that start with "p" but do NOT contain "sql".
# Syntax: ~(pattern)
# Requires flags: "COMPLEMENT"
# Note: The tilde ~ negates the *shortest following expression*.
q_regex_complement = {
    "regexp": {
        "_raw": {
            "value": "p.*~(.sql)",
            "flags": "COMPLEMENT"
        }
    }
}
print("\n--- REGEX: COMPLEMENT (Starts with 'p', NOT ending in 'sql') ---")
# This is complex and often tricky in Lucene.
search(q_regex_complement)

--- REGEX: BASIC (post.*[sl]) ---
⚡ Found 10000 hits in 1ms
--------------------------------------------------
[1] Score: 1.0 | ID: dbhpzJoBDLC7DRstIQVl
    Content: 2025-11-28 21:40:31 UTC [39]: [24092-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[2] Score: 1.0 | ID: drhpzJoBDLC7DRstIQVl
    Content: 2025-11-28 21:40:31 UTC [39]: [24093-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
--------------------------------------------------
[3] Score: 1.0 | ID: d7hpzJoBDLC7DRstIQVl
    Content: 2025-11-28 21:40:32 UTC [34]: [24097-1] user=postgres,db=postgres,app=Patroni restapi,client=127.0.0.1, e=00000 LOG:  statement: SELECT pg_catalog.pg_postmaster_start_time(), CASE WHEN pg_catalog.pg_i...
----------

## 15. Aggregations (Analytics)

Aggregations allow you to build complex data summaries and analytics. They are the "GROUP BY" and "SUM/AVG" of OpenSearch.

**The Three Pillars:**
1.  **Metric Aggregations**: Calculate metrics (sum, avg, min, max, stats) on a set of documents.
2.  **Bucket Aggregations**: Group documents into buckets (terms, ranges, histograms).
3.  **Pipeline Aggregations**: Perform calculations on the output of other aggregations (derivative, moving average).

**Structure:**
```json
"aggs": {
    "NAME_OF_AGGREGATION": {
        "TYPE": { ... params ... },
        "aggs": { ... sub-aggregations ... }
    }
}
```
**Tip:** Set `"size": 0` in the root query if you only care about the aggregation results and not the actual hits.

In [39]:
# --- DEMO: AGGREGATIONS ---

# 1. Metric Aggregation (Stats)
# Get min, max, avg, sum, and count of a numeric field (e.g., 'response_time' or 'bytes').
# We'll use 'bytes' if available, or just count docs if not.
# Let's assume we have a numeric field. If not, we'll use 'script' to fake one for demo.
q_agg_stats = {
    "size": 0, # Don't return hits
    "aggs": {
        "bytes_stats": {
            "stats": {
                "field": "bytes" # Replace with actual numeric field
            }
        }
    }
}
print("--- AGG: STATS (Min, Max, Avg, Sum) ---")
# Note: If 'bytes' doesn't exist, this returns all nulls/zeros.
search(body=q_agg_stats)

# 2. Bucket Aggregation (Terms)
# "GROUP BY host.name"
# Shows top 3 hosts by document count.
q_agg_terms = {
    "size": 0,
    "aggs": {
        "top_hosts": {
            "terms": {
                "field": "host.name",
                "size": 3
            }
        }
    }
}
print("\n--- AGG: TERMS (Top 3 Hosts) ---")
search(body=q_agg_terms)

# 3. Nested Aggregation (Terms -> Stats)
# "For each Host, calculate the Average Response Time"
# Group by host.name -> Then calculate stats on 'response_time' (simulated)
q_agg_nested = {
    "size": 0,
    "aggs": {
        "by_host": {
            "terms": {
                "field": "host.name",
                "size": 3
            },
            "aggs": {
                "avg_response_time": {
                    "avg": {
                        "script": {
                            "source": "doc['message.keyword'].value.length()" # Fake metric: message length
                        }
                    }
                }
            }
        }
    }
}
print("\n--- AGG: NESTED (Avg Message Length per Host) ---")
search(body=q_agg_nested)

# 4. Date Histogram (Time Series)
# "Count logs per hour"
q_agg_date = {
    "size": 0,
    "aggs": {
        "logs_over_time": {
            "date_histogram": {
                "field": "@timestamp",
                "fixed_interval": "1h"
            }
        }
    }
}
print("\n--- AGG: DATE HISTOGRAM (Logs per Hour) ---")
search(body=q_agg_date)

# 5. Cardinality (Distinct Count)
# "How many UNIQUE hosts are there?"
q_agg_cardinality = {
    "size": 0,
    "aggs": {
        "unique_host_count": {
            "cardinality": {
                "field": "host.name"
            }
        }
    }
}
print("\n--- AGG: CARDINALITY (Distinct Hosts) ---")
search(body=q_agg_cardinality)

--- AGG: STATS (Min, Max, Avg, Sum) ---
⚡ Found 10000 hits in 11ms
--------------------------------------------------

--- AGG: TERMS (Top 3 Hosts) ---
Error: 400 Client Error: Bad Request for url: http://localhost:19200/patronidata/_search
{"error":{"root_cause":[{"type":"illegal_argument_exception","reason":"Text fields are not optimised for operations that require per-document field data like aggregations and sorting, so these operations are disabled by default. Please use a keyword field instead. Alternatively, set fielddata=true on [host.name] in order to load field data by uninverting the inverted index. Note that this can use significant memory."}],"type":"search_phase_execution_exception","reason":"all shards failed","phase":"query","grouped":true,"failed_shards":[{"shard":0,"index":"patronidata","node":"PRbPvKIxTvq8dUFkBwZE9Q","reason":{"type":"illegal_argument_exception","reason":"Text fields are not optimised for operations that require per-document field data like aggregati